In [101]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import stalse_functions as sf
import pandas_gbq


In [126]:
def get_facebook_insights(page_id, access_token, metric, start_date, end_date):
    url = f'https://graph.facebook.com/v20.0/{page_id}/insights'
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    start = (start + relativedelta(day=31))
    end = (end + relativedelta(day=31))
    
    all_data = []
    current = start
    
    while current < end:
        batch_end = (current + relativedelta(months=1, day=31))
        batch_end = min(batch_end, end)
        params = {
            'access_token': access_token,
            'metric': metric,
            'period': 'day',
            'since': current.strftime('%Y-%m-%d'),
            'until': batch_end.strftime('%Y-%m-%d')
        }
        print(f"Making API call for period: {current.strftime('%Y-%m-%d')} to {batch_end.strftime('%Y-%m-%d')}")
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error in API call: {response.status_code}")
            print(f"Response content: {response.text}")
            return pd.DataFrame()
        data = response.json()
        if 'data' in data and data['data']:
            all_data.extend(data['data'][0]['values'])
        else:
            print(f"No data returned for period: {current.strftime('%Y-%m-%d')} to {batch_end.strftime('%Y-%m-%d')}")
        current = batch_end
    if not all_data:
        print("No data collected across all API calls")
        return pd.DataFrame()
    df = pd.DataFrame(all_data)
    if 'end_time' in df.columns:
        df['end_time'] = pd.to_datetime(df['end_time'])
    
    return df

In [139]:
page_id = '151189004972368'
access_token = 'EAAPGyUspGX4BO3iqr4N8UJotTjnDRboYbpFZCXvzb7zt9QmWHZAHYzZClyRtoBe98zBaHL34IEFvA0TchaFtIVkqQOJEflrDZCdnFCZBHqRJUp8dN9llD4EGJDejWUQBFxWi19GKWvGtJugASEcPZBpJsEDAuMxZAxsmCjPtySBux8jEEkXZC2o7qK5SeSBnLGQZCEAX5vOHTmBCmiNOpLNB3cnAzOFocqonF'
start_date = '2023-08-31'
end_date = '2024-09-12'
metric = 'page_follows'

In [ ]:
df = get_facebook_insights(page_id, access_token, metric, start_date, end_date)


In [129]:
df = df.rename(columns={'value': 'likes', 'end_time': 'date'})


In [130]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [133]:
df['date'] = pd.to_datetime(df['date'])

In [134]:
cutoff_date = pd.Timestamp('2024-09-12')

In [ ]:
filtered_df = df[df['date'] <= cutoff_date]
filtered_df['date'] = filtered_df['date'].dt.strftime('%Y-%m-%d')

In [ ]:
filtered_df

In [ ]:
sf.cria_bq(df, 'panasonic_facebook.likes', 'replace', 'repense-398114')